In [2]:
import pandas as pd


In [25]:
airfoil_res= pd.read_csv("airfoil/results.csv")
airfoil_res.groupby("model")[["rmse","nll","runtime_s"]].mean()

,rmse,nll,runtime_s
model,,,
bde,0.133265,-0.828248,374.425833
catboost,0.218369,-0.102722,1.183444
linear,0.686781,1.043199,0.001001
rf,0.265208,0.091640,0.938868
tabpfn,0.144006,-0.518960,8.269187
xgb,0.199536,-0.193168,1.043612


In [26]:
bikesharing_res= pd.read_csv("bikesharing/results.csv")
bikesharing_res.groupby("model")[["rmse","nll","runtime_s"]].mean()

,rmse,nll,runtime_s
model,,,
bde,0.226962,-0.730670,1696.934843
catboost,0.226876,-0.064423,2.213817
linear,0.779625,1.169997,0.004100
rf,0.244070,0.008638,17.428404
tabpfn,0.211832,-0.133024,821.031543
xgb,0.214973,-0.118323,2.472263


In [27]:
import inspect
from tabpfn import TabPFNRegressor
print(inspect.signature(TabPFNRegressor.predict))
print(TabPFNRegressor.predict.__doc__)


(self, X: 'XType', *, output_type: 'OutputType' = 'mean', quantiles: 'list[float] | None' = None) -> 'RegressionResultType'
Runs the forward() method and then transform the logits
        from the binning space in order to predict target variable.

        Args:
            X: The input data.
            output_type:
                Determines the type of output to return.

                - If `"mean"`, we return the mean over the predicted distribution.
                - If `"median"`, we return the median over the predicted distribution.
                - If `"mode"`, we return the mode over the predicted distribution.
                - If `"quantiles"`, we return the quantiles of the predicted
                    distribution. The parameter `quantiles` determines which
                    quantiles are returned.
                - If `"main"`, we return the all output types above in a dict.
                - If `"full"`, we return the full output of the model, including the
        

In [30]:
from catboost import CatBoostRegressor
print(inspect.signature(CatBoostRegressor.predict))
print(CatBoostRegressor.predict.__doc__)
print(hasattr(CatBoostRegressor, "virtual_ensembles_predict"))
print(inspect.signature(CatBoostRegressor.virtual_ensembles_predict))

(self, data, prediction_type=None, ntree_start=0, ntree_end=0, thread_count=-1, verbose=None, task_type='CPU')

        Predict with data.

        Parameters
        ----------
        data : catboost.Pool or list of features or list of lists or numpy.ndarray or pandas.DataFrame or pandas.Series
                or catboost.FeaturesData
            Data to apply model on.
            If data is a simple list (not list of lists) or a one-dimensional numpy.ndarray it is interpreted
            as a list of features for a single object.

        prediction_type : string, optional (default='RawFormulaVal')
            Can be:
            - 'RawFormulaVal' : return raw formula value.
            - 'Exponent' : return Exponent of raw formula value.

        ntree_start: int, optional (default=0)
            Model is applied on the interval [ntree_start, ntree_end) (zero-based indexing).

        ntree_end: int, optional (default=0)
            Model is applied on the interval [ntree_start, n

In [31]:
import argparse
import os
import sys


from bde import BdeRegressor
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
import jax.numpy as jnp
from sklearn.neural_network import MLPRegressor
from bde import BdeRegressor
from sklearn.linear_model import LinearRegression
import time
from pathlib import Path
from bde.loss.loss import GaussianNLL
from bde.sampler.prior import PriorDist
import logging
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from tabpfn import TabPFNRegressor
from tabpfn.constants import ModelVersion
from catboost import CatBoostRegressor

In [35]:
DATA_DIR = "../bde/data"
DATASETS = {
    "airfoil": {"file": "airfoil.csv", "sep": ",", "header": 0},
    "concrete": {"file": "concrete.data", "sep": " ", "header": None},
    "bikesharing": {"file": "bike_sharing_dataset/hour.csv", "sep": ",", "header": 0},
}


def config_data(dataset, split_seed: int):
    spec = DATASETS[dataset]
    path = DATA_DIR +"/"+ spec["file"]
    data = pd.read_csv(path, sep=spec["sep"], header=spec["header"])

    if dataset == "bikesharing":
        y = data["cnt"].to_numpy(dtype=float)

        X = data.drop(
            columns=["cnt", "casual", "registered", "instant", "dteday"]
        ).to_numpy(dtype=float)
    else:
        X = data.iloc[:, :-1].to_numpy(dtype=float)
        y = data.iloc[:, -1].to_numpy(dtype=float)

    X = jnp.array(X)
    y = jnp.array(y)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=split_seed
    )
    x_scaler = StandardScaler()
    y_scaler = StandardScaler()

    X_train = x_scaler.fit_transform(X_train)
    X_test = x_scaler.transform(X_test)

    y_train = y_scaler.fit_transform(y_train.reshape(-1, 1)).ravel()
    y_test = y_scaler.transform(y_test.reshape(-1, 1)).ravel()

    return X_train, X_test, y_train, y_test

def catboost_reg(seed) -> CatBoostRegressor:
    return CatBoostRegressor(
        random_seed=seed,
        depth=6,
        iterations=1000,
        learning_rate=0.05,
        allow_writing_files=False,
        verbose=False,
    )

X_train, X_test, y_train, y_test = config_data("airfoil", split_seed=0)


In [ ]:
model =catboost_reg(0)
model.fit(X_train, y_train)


In [52]:
virtual_ensembles = model.virtual_ensembles_predict(X_test)
print(virtual_ensembles.shape)
print(virtual_ensembles[0,...].shape)
print(model.predict(X_test).shape)

(301, 10, 1)
(10, 1)
(301,)


In [49]:
def bde_regressor(seed) -> BdeRegressor:
    return BdeRegressor(
        n_members=10,
        hidden_layers=[4, 8, 8, 8, 8, 4],
        loss=GaussianNLL(),
        epochs=100,
        validation_split=0.15,
        lr=1e-2,
        weight_decay=1e-4,
        warmup_steps=500,
        n_samples=10,
        n_thinning=2,
        patience=20,
        seed=seed,
    )

In [50]:
bde = bde_regressor(0)
bde.fit(X_train,y_train)

MCLMC warmup: 100%|██████████| 500/500 [00:09<00:00, 54.02it/s] 


,n_members,10
,hidden_layers,"[4, 8, ...]"
,seed,0
,loss,<bde.loss.los...001B7D3878990>
,activation,'relu'
,epochs,100
,patience,20
,validation_split,0.15
,n_samples,10
,warmup_steps,500
,lr,0.01


In [51]:
mu,sigma = bde.predict(X_test, mean_and_std=True)
print(mu.shape, sigma.shape)

(301,) (301,)


In [53]:
def tabpfn_reg(seed) -> TabPFNRegressor:
    return TabPFNRegressor.create_default_for_version(
        ModelVersion.V2, ignore_pretraining_limits=True, random_state=seed
    )

In [58]:
tabpfn = tabpfn_reg(0)
tabpfn.fit(X_train,y_train)


,n_estimators,8
,categorical_features_indices,None
,softmax_temperature,0.9
,average_before_softmax,False
,model_path,'C:\\Users\\vyron\\AppD...abpfn-v2-regressor.ckpt'
,device,'auto'
,ignore_pretraining_limits,True
,inference_precision,'auto'
,fit_mode,'fit_preprocessors'
,memory_saving_mode,'auto'
,random_state,0


In [ ]:
out = tabpfn.predict(X_test, output_type="main", quantiles=[0.25, 0.75])
print(out.keys())



dict_keys(['mean', 'median', 'mode', 'quantiles'])


In [ ]:
mu = out["mean"]
qs = out["quantiles"]
print(mu.shape)
print(type(qs), len(qs))
print(qs[0].shape, qs[1].shape)
qs_1, qs_2 = qs
print(qs_1.shape, qs_2.shape)


(301,)
(301,)
<class 'list'> 2
(301,) (301,)
(301,) (301,)
